# HOMER DATA PROCESSING SCRIPT
The goal of this script is to take the output file of HOMER Pro Batch Runs (.xlsx file containing one optimal case/sheet) and produce a condensed data set + plots.

General flow:
1) Run HOMER study via batch process. Select "export results" and name the file.


2) Import file to this script.


3) Use Pandas to pull .xlsx data into a more usable format. 


4) Use collected data to build plots. 

Goals:


Have this script take a large HOMER study (>100 cases) and build one easily accessible database.


Potentially couple this script with a HOMER input reading script to do cartography w/HOMER geography + results.


In [30]:
# Block of dependencies and working directory
import matplotlib
import conda
import numpy
import pandas as pd
import os 
directory = 'd:/repo/homer/ClimateStudies/OutputProcessing/Data'
#cwd = os.path.abspath('./Data')
#display(cwd)
#files = os.listdir(directory)
#display(files)
#merged = pd.DataFrame()
##merging everything
#for file in files:
#    display(file)
#    if file.endswith(".xlsx"):
#        merged = merged.append(pd.read_excel(file),ignore_index=true) 

## Following this guide for xlsx reading
https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook

In [31]:
#output = pd.read_excel('RevisedCase1.xlsx',sheet_name=None)#brings out all of the xlsx data as a dictionary
#output_keys = output.keys()  #grabs dictionary keys 
#type(output_keys)
#print(output_keys)
#data = output[output_keys]
#data["Cost/NPC\n($)"]

In [32]:
# reads, maps all sheets 
files = os.listdir(directory)
display(files)
output_df = {} #creates empty dictionary
names = [] #creates list of empty dict
for file in files:
    if file.endswith('.xlsx'):
        xls = pd.ExcelFile(directory + '/' + file) #reads file
        names = names + xls.sheet_names
        for sheet_name in xls.sheet_names: #pulls individual sheets
            output_df[sheet_name] = xls.parse(sheet_name) #puts each sheet into a dictionary w/key "sheet_name"

['FullCase1.xlsx', 'FullCase2.xlsx', 'FullCase3.xlsx', 'FullCase4.xlsx']

In [35]:
#pulling interesting data points from each 
opt_out = pd.DataFrame(columns=['Architecture/PV\n(kW)', 'Architecture/G1500', 'Architecture/1kWh LI',
                                                 'Cost/COE\n($)', 'Architecture/GenLarge(kW)'])
n = len(names)
for i in range(n):
    name = names[i] # Grabs Sheet Name
    Location = name[5:-3]
    case_num = name[:5] # Grabs Case Number 
    KCC = name[-3:]
    KCC = KCC.lower()
    filtered_out = output_df[name].filter(['Architecture/PV\n(kW)', 'Architecture/G1500', 'Architecture/1kWh LI',
                                                 'Cost/COE\n($)', 'Architecture/GenLarge(kW)']) #Filtering objects of interest
    filtered_out['CaseName'] = Location #adding actual case name to series
    filtered_out['CaseNum'] = case_num # adding case num to series
    filtered_out['KCC'] = KCC
    framed = filtered_out.iloc[[0]] #making the actual df
    framed.set_index(['CaseName','CaseNum'], inplace=True, drop=True) #building df w/ casename and num
    opt_out = pd.concat([framed,opt_out]) #concating all DFs together
opt_out.to_excel("LocationalVarianceTrimmed.xlsx")
display(opt_out)

,,Architecture/PV\n(kW),Architecture/G1500,Architecture/1kWh LI,Cost/COE\n($),KCC,Architecture/GenLarge(kW)
CaseName,CaseNum,,,,,,
WadenaMN,Case4,10727.029540,7.0,25372,0.138638,dfa,NaN
VegasNV,Case4,19857.025383,6.0,43517,0.155043,bwh,NaN
UPMich,Case4,17200.157069,9.0,37649,0.162512,dfb,NaN
TexasOil,Case4,14815.152633,6.0,35220,0.131990,bsh,NaN
Syracuse,Case4,33010.742566,6.0,32103,0.179424,dfb,NaN
...,...,...,...,...,...,...,...
BendOre,Case1,36672.913375,23.0,97456,0.233693,bsk,NaN
BangorME,Case1,61149.251524,22.0,138129,0.302428,dfb,NaN
AtlantaGA,Case1,93091.971417,3.0,121562,0.252569,cfa,NaN
